# Generate Mock Data

In [61]:
import numpy as np
import matplotlib.pyplot as plt
import random
from random import uniform
import database.config as config
import os

### (1) Generate Spike Times

In [37]:
patient_id = 1
session_nr = 1

nr_units_patient_1 = 100
begin_recording_time = 449860058
stop_recording_time = 454664467

unit_types = ['M', 'S']

In [3]:
spikes = []
for i in range(nr_units_patient_1):
    nr_spikes = int(uniform(10000, 16000))
    spike_vec = []
    for j in range(nr_spikes):
        spike_vec.append(uniform(begin_recording_time, stop_recording_time))
    spikes.append(spike_vec)

In [44]:
for i in range((nr_units_patient_1)):
    np.save("{}/mock_data/spikes/{}/session_{}/CSC{}_{}UA{}.npy".format(config.PATH_TO_REPO, patient_id, session_nr, i+1, random.choice(unit_types), '1'), spikes[i])

In [32]:
patient_id = 2
session_nr = 1
nr_units_patient_2 = 84 
begin_recording_time = 449860058
stop_recording_time = 454664467

spikes = []
for i in range(nr_units_patient_2):
    nr_spikes = int(uniform(10000, 16000))
    spike_vec = []
    for j in range(nr_spikes):
        spike_vec.append(uniform(begin_recording_time, stop_recording_time))
    spikes.append(spike_vec)
    
for i in range((nr_units_patient_2)):
    np.save("{}/mock_data/spikes/{}/session_{}/CSC{}_{}UA{}.npy".format(config.PATH_TO_REPO, patient_id, session_nr, i+1, random.choice(unit_types), '1'), spikes[i])

In [33]:
patient_id = 3
session_nr = 1
nr_units_patient_3 = 59
begin_recording_time = 449860058
stop_recording_time = 454664467

spikes = []
for i in range(nr_units_patient_3):
    nr_spikes = int(uniform(10000, 16000))
    spike_vec = []
    for j in range(nr_spikes):
        spike_vec.append(uniform(begin_recording_time, stop_recording_time))
    spikes.append(spike_vec)
    
for i in range((nr_units_patient_3)):
    np.save("{}/mock_data/spikes/{}/session_{}/CSC{}_{}UA{}.npy".format(config.PATH_TO_REPO, patient_id, session_nr, i+1, random.choice(unit_types), '1'), spikes[i])

### (2) Create Channel Names File

In [77]:
# For patient 1
brain_regions = ["LA", "LAH", 'LEC', "LMH", "LPHC", "RA", "RAH", "REC", "RMH", "RPCH"]
nr_units_per_brain_region_1 = [7, 12, 9, 10, 15, 13, 8, 11, 12, 3]

print(sum(nr_units_per_brain_region_1) == nr_units_patient_1)

if os.path.exists("{}/mock_data/session_data/session_1_1_20190101_12h00m00s/ChannelNames.txt".format(config.PATH_TO_REPO)):
    os.remove("{}/mock_data/session_data/session_1_1_20190101_12h00m00s/ChannelNames.txt".format(config.PATH_TO_REPO))
    
f1 = open("{}/mock_data/session_data/session_1_1_20190101_12h00m00s/ChannelNames.txt".format(config.PATH_TO_REPO), 'w+')
for i in range(len(nr_units_per_brain_region_1)):
    for j in range(nr_units_per_brain_region_1[i]):
        f1.write("{}{}.ncs\n".format(brain_regions[i], j+1))
        
f1.close()

True


In [78]:
# for patient 2
nr_units_per_brain_region_2 = [12, 9, 10, 5, 7, 11, 8, 7, 12, 3]

if os.path.exists("{}/mock_data/session_data/session_2_1_20160101_12h00m00s/ChannelNames.txt".format(config.PATH_TO_REPO)):
    os.remove("{}/mock_data/session_data/session_2_1_20160101_12h00m00s/ChannelNames.txt".format(config.PATH_TO_REPO))

f2 = open("{}/mock_data/session_data/session_2_1_20160101_12h00m00s/ChannelNames.txt".format(config.PATH_TO_REPO), 'w+')
for i in range(len(nr_units_per_brain_region_2)):
    for j in range(nr_units_per_brain_region_2[i]):
        f2.write("{}{}.ncs\n".format(brain_regions[i], j+1))
        
f2.close()

In [79]:
# for patient 3
nr_units_per_brain_region_3 = [6, 5, 4, 6, 7, 10, 3, 7, 5, 6]
sum(nr_units_per_brain_region_3)

if os.path.exists("{}/mock_data/session_data/session_3_1_20170101_12h00m00s/ChannelNames.txt".format(config.PATH_TO_REPO)):
    os.remove("{}/mock_data/session_data/session_3_1_20170101_12h00m00s/ChannelNames.txt".format(config.PATH_TO_REPO))

f3 = open("{}/mock_data/session_data/session_3_1_20170101_12h00m00s/ChannelNames.txt".format(config.PATH_TO_REPO), 'w+')
for i in range(len(nr_units_per_brain_region_3)):
    for j in range(nr_units_per_brain_region_3[i]):
        f3.write("{}{}.ncs\n".format(brain_regions[i], j+1))
        
f3.close()

### (2) Create Events File

In [ ]:
len_context_files = random.randint(100, 200) # generate length of events.nev & DAQ file. 

# recreate pings
if len_context_files % 8 == 0:
    reps = len_context_files / 8
else:
    reps = int(len_context_files / 8) + 1
    
signal_tile = np.tile([1,2,4,8,16,32,64,128], reps)
signal_tile = signal_tile[:len_context_files]

# recreate event timestamps
events = np.linspace(begin_recording_time, stop_recording_time, num=len_context_files)

events_mat = list(zip(events, signal_tile))

### (3) Create DAQ File

### (4) Create Watch Log File

### (5) Create Stimulus Meta Data (here movie annotation)

In [81]:
nr_movie_frames = 125725      # movie length: 5029 seconds (AVI file); 5029/0.04 = 125725
perfect_pts = [round((x * 0.04), 2) for x in range(1, nr_movie_frames+1)]  

annotator_ids = []
for i in range(len(config.annotators)):
    annotator_ids.append(config.annotators[i]['annotator_id'])

path_to_movie_annotations = "mock_data/movie_annotation"

nr_character_labels = 2

start_times_1 = [0, 5000.04, 7000.04, 12000.04]
stop_times_1 = [5000,7000,12000,12575]
values_1 = [1,0,1,0]
character1 = np.array([start_times_1, stop_times_1, values_1]) 
np.save("{}/{}/{}_character1_{}_20191212_character.npy".format(
    config.PATH_TO_REPO, path_to_movie_annotations, 1, random.choice(annotator_ids)), character1)


start_times_2 = [0, 400.04, 4000.04, 10000.04, 10500.04]
stop_times_2 = [400,4000,10000,10500,12575]
values_2 = [0,1,0,1,0]
character2 = np.array([start_times_2, stop_times_2, values_2]) 
np.save("{}/{}/{}_character2_{}_20191010_character.npy".format(
    config.PATH_TO_REPO, path_to_movie_annotations, 2, random.choice(annotator_ids)), character2)

start_times_3 = [0, 100.04, 500.04]
stop_times_3 = [100, 500, 12575]
values_3 = [0,1,0]
annot4 = np.array([start_times_3, stop_times_3, values_3]) 
np.save("{}/{}/{}_location1_{}_20200101_location.npy".format(
    config.PATH_TO_REPO, path_to_movie_annotations, 3, random.choice(annotator_ids)), annot4)
